In [ ]:
# Establecer conexion 
import openeo
#nos contectamos a la plataforma
connection = openeo.connect("https://openeo.dataspace.copernicus.eu").authenticate_oidc()
print(connection.list_collections())
print("conexion exitosa")
print(f"conectado a {connection.title()} como  '{connection.user_id()}")    

In [ ]:
# Descargar los .tif para un periodo de 6 meses. 
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from datetime import date 

# coordenadas de los lagos 
lago_atitlan = {
    "west": -91.326256, "east": -91.07151, "south": 14.5948, "north": 14.750979
}
lago_amatitlan = {
    "west": -90.638065, "east": -90.512924, "south": 14.412347, "north": 14.493799
}

#definimos el periodo de observacion, 6 meses -> enero a junio 2025
periodo_observacion = ['2025-01-01', '2025-07-01']

#bandas necesarias para el analisis de cianobacterias, incluimos visibles NIR y red-edge
bandas_necesarias = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08']

#cargamos el cubo de datos para el lago de atitlan 
print(f"cargando datos para el lago de atitlan")
atitlan_cube = connection.load_collection(
    "SENTINEL2_L2A", 
    spatial_extent=lago_atitlan, 
    temporal_extent=periodo_observacion,
    bands=bandas_necesarias
)

#Guardamos los resultado en formato GeoTIFF, el resultado sera un datacube, un tif con multiples capas, una por banda y por fecha 
result_graph = atitlan_cube.save_result(format="GTiff")

#crear y ejecuta el trabajo, tarea de procesamiento en la nube 
print("creando y ejecutando el trabajo en copernicuso")
job = connection.create_job(result_graph, title=" Descargar Lago Atitlan - Cianobacterias")
job.start_and_wait()

# descarga los resultados 
print("Descargaando los resultados")
job.download_results("./Bandas_Atitlan_6meses.tif")
print("Descarga para el lago de atitlan completada")


# **Repetir el proceso para el Lago de Amatitlán**
# print("\nCargando datos para el Lago de Amatitlán...")
# amatitlan_cube = connection.load_collection(
#     "SENTINEL2_L2A",
#     spatial_extent=lago_amatitlan,
#     temporal_extent=periodo_observacion,
#     bands=bandas_requeridas
# )
# result_graph_amatitlan = amatitlan_cube.save_result(format="GTIFF")
# job_amatitlan = connection.create_job(result_graph_amatitlan, title="DescargaAmatitlanCiano")
# job_amatitlan.start_and_wait()
# job_amatitlan.download_results("./Bandas_Amatitlan_6meses.tif")
# print("¡Descarga para el Lago de Amatitlán completada!")
